# Module 02: Experiment Tracking

The purpose of this notebook is to follow the second module and its videos:
- Getting started with MLflow
- Experiment tracking with MLflow
- Model Management

The base for this notebook is the one from the Module 01: Introduction homework, and its video.

In [1]:
# Import packages for preparing and training data
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

In [2]:
# Import MLFlow for experiment tracking
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

# Import pickle to save the model
import pickle

In [3]:
# Import xgb and hyperopt

import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [10]:
# Create a function to prepare taxi data
def prepare_taxi_data(file):
    
    # Read the parquet file
    df = pd.read_parquet(file)

    # Create duration column
    df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
    df["duration"] = pd.to_timedelta(df["duration"])/pd.Timedelta('60s')

    # Remove outliers
    outliers_indices = np.where((df["duration"] < 1) | (df["duration"] > 60))
    df = df.drop(outliers_indices[0])

    # Change categorical columns to string to not generate conflicts later when
    # creating the features matrix
    categorical=['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2585010344.py, line 11)

In [6]:
# Prepare training data
training_df = prepare_taxi_data('../../../data/yellow_tripdata_2023-01.parquet')

# Prepare validation data
validate_df = prepare_taxi_data('../../../data/yellow_tripdata_2023-02.parquet')

In [7]:
# One-hot encode and get features matrix
dv = DictVectorizer()

# Learn mappings from dict and transform to matrix
train_dicts = training_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

# Learn mappings from dict and transform to matrix
val_dicts = validate_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
# Prepare target values to train the model
y_train = training_df['duration'].values
y_val = validate_df['duration'].values

In [14]:
# Option number one to save models

# Start a new MLfLow run, train the Lasso model and save it

with mlflow.start_run():
    # Log tags and params
    mlflow.set_tag('developer', 'eduga')
    mlflow.set_tag('model', 'lasso')
    mlflow.log_param('train_data_path', '../../../data/yellow_tripdata_2023-01.parquet')
    mlflow.log_param('validate_data_path', '../../../data/yellow_tripdata_2023-02.parquet')
    
    alpha = 0.01
    mlflow.log_param('alpha', alpha)

    # Train the model and predict
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)

    # Get the error and log it
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)

    # Save the preprocessor
    with open('models/preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact(
        'models/preprocessor.b',
        artifact_path='preprocessor'
    )

    # Save the model
    mlflow.sklearn.log_model(
        lr,
        artifact_path='models_mlflow'
    )

In [11]:
# Option number two to save models

# Model the best xgb run from mlflow, performance and optimization wise
best_params = {
    'learning_rate': 0.7720684014557972,
    'max_depth': 58,
    'min_child_weight': 0.6989693091372148,
    'objective': 'reg:squarederror',
    'reg_alpha': 0.015100098245236579,
    'reg_lambda': 0.03214589450610109,
    'seed': 42
}

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

with mlflow.start_run():
    # Log tags and params
    mlflow.set_tag('model', 'xgboost')
    mlflow.set_tag('developer', 'eduga')
    mlflow.log_param('train_data_path', '../../../data/yellow_tripdata_2023-01.parquet')
    mlflow.log_param('validate_data_path', '../../../data/yellow_tripdata_2023-02.parquet')
    mlflow.log_params(best_params)

    # Train the model and predict
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=250,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )
    y_pred = booster.predict(valid)

    # Get the error and log it
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    # Save and log preprocessor
    with open('models/preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact(
        'models/preprocessor.b',
        artifact_path='preprocessor'
    )
        
    # Use xgboost.log_model method to save the model
    mlflow.xgboost.log_model(
        booster,
        artifact_path='models_mlflow'
    )

[0]	validation-rmse:8.08909
[1]	validation-rmse:6.36828
[2]	validation-rmse:6.11740
[3]	validation-rmse:5.85362
[4]	validation-rmse:5.80118
[5]	validation-rmse:5.55971
[6]	validation-rmse:5.45833
[7]	validation-rmse:5.42895
[8]	validation-rmse:5.38311
[9]	validation-rmse:5.33248
[10]	validation-rmse:5.29358
[11]	validation-rmse:5.28172
[12]	validation-rmse:5.27465
[13]	validation-rmse:5.26293
[14]	validation-rmse:5.23730
[15]	validation-rmse:5.23484
[16]	validation-rmse:5.23284
[17]	validation-rmse:5.22938
[18]	validation-rmse:5.22391
[19]	validation-rmse:5.22091
[20]	validation-rmse:5.21904
[21]	validation-rmse:5.21889
[22]	validation-rmse:5.21787
[23]	validation-rmse:5.21678
[24]	validation-rmse:5.21479
[25]	validation-rmse:5.21395
[26]	validation-rmse:5.21387
[27]	validation-rmse:5.21391
[28]	validation-rmse:5.21365
[29]	validation-rmse:5.21339
[30]	validation-rmse:5.21337
[31]	validation-rmse:5.21336
[32]	validation-rmse:5.21324
[33]	validation-rmse:5.21324
[34]	validation-rmse:5.2

In [12]:
# In MLflow, when inspecting a run's artifacts, the run's URI can be extracted. With it,
# a new xgb booster object can be made. This can also be a python function, where mlflow
# and pandas are the only dependencies to be imported.

# URI from MLflow
logged_model = 'runs:/cbf3d520b4d44905bd9c8cc155140711/models_mlflow'

# Apply the same framework we've been iterating
xgboost_model = mlflow.xgboost.load_model(logged_model)
y_pred = xgboost_model.predict(valid)
y_pred[:5]

array([ 6.290758, 44.159   , 15.870378, 21.573727, 25.2379  ],
      dtype=float32)